# pySCENIC - including repressors, post analysis

Create a list of repressors and count the number of runs they appeared at as modules and then regulons!

In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from glob import glob
import pandas as pd
import pickle

In [3]:
results_dir_glob = "/Users/danabarilan/Documents/forMasters/Intenship/code/results_2709/run_*/"
results_dir = "/Users/danabarilan/Documents/forMasters/Intenship/code/results_2709/"

In [19]:
all_runs_repressors = pd.DataFrame()
for folder in glob(results_dir_glob):
    with open(folder + 'modules.pkl', 'rb') as f:
        modules = pickle.load(f)

    repressors_list = [mod for mod in modules 
                       if 'repressing' in mod.context]  # only the repressing moduels list
    
    repressors_df = pd.DataFrame({'TF': [rep.transcription_factor for rep in repressors_list],
                                  'gene': [rep.gene2weight.keys() for rep in repressors_list]})
    repressors_df = repressors_df.explode('gene').drop_duplicates()
    # concat all results together
    all_runs_repressors = pd.concat([all_runs_repressors, repressors_df])

all_runs_repressors = all_runs_repressors.groupby(['TF','gene']).size().to_frame('count')
all_runs_repressors = all_runs_repressors.sort_values(['TF', 'count'], ascending=False)

all_runs_repressors = all_runs_repressors.reset_index().assign(TF=lambda x: x['TF'].where(~x['TF'].duplicated(), ''))
all_runs_repressors.to_csv(results_dir+'repressors_counts.csv', index='False')
    

In [15]:
display(all_runs_repressors)

TF      gene   
CG4440  Act57B      1
        Act87E      2
        BBS8        1
        CG11658     1
        CG14207     2
                   ..
sna     repo        2
        sna        11
        spidey      1
        unpg        1
        uzip        8
Length: 2792, dtype: int64